<h3><span style="color:red"> Note: <b>%%bash</b> blocks does not show intermediate outputs, you may want to copy and run these in terminal </span></h3>

# Prepare Data

## Download

Oxford-IIT Pets Dataset https://www.robots.ox.ac.uk/~vgg/data/pets/

In [ ]:
%%bash
mkdir -p /data/pets
cd /data/pets

export IMAGE_URL='https://www.robots.ox.ac.uk/~vgg/data/pets/data/images.tar.gz'  
export LABEL_URL='https://www.robots.ox.ac.uk/~vgg/data/pets/data/annotations.tar.gz'
if [[ -d images ]] && [[ $(ls images/*.jpg | wc -l) -ge 7390 ]];then  
  echo "No need to Download"
  echo "Images already there in ${PWD}/training/image_2/"
else
  echo "Starting Download" 
  wget -c --show-progress --progress=bar ${IMAGE_URL}
  wget -c --show-progress --progress=bar ${LABEL_URL}
  echo "Extracting" 
  tar -xf images.tar.gz
  tar -xf annotations.tar.gz
fi


## Validate

In [ ]:
%%bash

cd /data/pets/

echo -n "Images: " ; ls images/*.jpg | wc -l # Should be # 7390

echo "Structure"
tree --filelimit 6

## Generate TF Record

In [ ]:
%%bash
# !ls ${TF_OBJECT_DETECTION_API}/data
export TFRECORD_PATH='/data/pets/tfrecord'
#export TFRECORD_PATH='/data/pets/tfrecord_mask'

if [ ! -d ${TFRECORD_PATH} ] 
then
    mkdir -p ${TFRECORD_PATH}
    python ${TF_OBJECT_DETECTION_API}/dataset_tools/create_pet_tf_record.py \
        --label_map_path=${TF_OBJECT_DETECTION_API}/data/pet_label_map.pbtxt \
        --data_dir=/data/pets \
        --output_dir=${TFRECORD_PATH} \
        --faces_only=true
fi

echo -n "TF Records: " ; ls ${TFRECORD_PATH} | wc -l 

# Prepare Model
## Downloading a COCO-pretrained Model for Transfer Learning

In [ ]:
%%bash

#export MODEL_NAME='ssd_mobilenet_v1_coco_2018_01_28'
export MODEL_NAME='faster_rcnn_resnet101_coco_2018_01_28'

mkdir -p /data/pretrained_models
cd /data/pretrained_models

if [[ -d /data/pretrained_models/${MODEL_NAME} ]]
then 
  echo "Model ${MODEL_NAME} already exists" 
else
  echo "Downloading model ${MODEL_NAME}" 
  wget -c --show-progress --progress=bar http://download.tensorflow.org/models/object_detection/${MODEL_NAME}.tar.gz 
  tar xvzf ${MODEL_NAME}.tar.gz
fi

echo "Model path: /data/pretrained_models/${MODEL_NAME}"

# Training
## Configure pipeline
Set the paths `PATH_TO_BE_CONFIGURED` to correct values

In [ ]:
!ls ${TF_OBJECT_DETECTION_API}/samples/configs/*pets.config

In [ ]:
%%bash
export MODEL_NAME='ssd_mobilenet_v1_coco_2018_01_28'
#export MODEL_NAME='faster_rcnn_resnet101_coco_2018_01_28'
export NUM_CLASS=37
export PRETRAINED_MODEL_PATH="/data/pretrained_models/${MODEL_NAME}"
export TFRECORD_PATH='/data/pets/tfrecord' # http://download.tensorflow.org/models/object_detection/pet_faces_tfrecord.tar.gz
export MODEL_OUT_PATH="/data/pets/trained_model"
mkdir -p ${MODEL_OUT_PATH}

#cp ${TF_OBJECT_DETECTION_API}/samples/configs/faster_rcnn_resnet101_pets.config ${MODEL_OUT_PATH}/{MODEL_NAME}_pipeline.config
cp ${PRETRAINED_MODEL_PATH}/pipeline.config ${MODEL_OUT_PATH}/pipeline.config

sed -i "s|PATH_TO_BE_CONFIGURED/model.ckpt|${PRETRAINED_MODEL_PATH}/model.ckpt|g" ${MODEL_OUT_PATH}/pipeline.config
sed -i "s|PATH_TO_BE_CONFIGURED/[a-z]*_label_map.pbtxt|${TF_OBJECT_DETECTION_API}/data/pet_label_map.pbtxt|g" ${MODEL_OUT_PATH}/pipeline.config
sed -i "s|PATH_TO_BE_CONFIGURED/[a-z]*_train|${TFRECORD_PATH}/pet_faces_train|g" ${MODEL_OUT_PATH}/pipeline.config
sed -i "s|PATH_TO_BE_CONFIGURED/[a-z]*_val|${TFRECORD_PATH}/pet_faces_val|g" ${MODEL_OUT_PATH}/pipeline.config
sed -i "s|num_classes: [[:digit:]]\+|num_classes: ${NUM_CLASS}|g" ${MODEL_OUT_PATH}/pipeline.config

sed -i "s|.record\"|.record-?????-of-00010\"|g" ${MODEL_OUT_PATH}/pipeline.config 
sed -i "s|step: 0|step: 1|g" ${MODEL_OUT_PATH}/pipeline.config # Bug fix 

# Check the final config manually
cat ${MODEL_OUT_PATH}/pipeline.config

## Setup Tensorboard

In [ ]:
%load_ext tensorboard.notebook
%tensorboard --logdir /data/pets/trained_model/

#from tensorboard import notebook
#notebook.display(height = 1000)

## Evalute pretrained model

In [ ]:
%env MODEL_NAME=ssd_mobilenet_v1_coco_2018_01_28
#export MODEL_NAME='ssd_mobilenet_v1_coco_2018_01_28'
!python ${TF_OBJECT_DETECTION_API}/model_main.py \
  --pipeline_config_path=/data/pets/trained_model/pipeline.config \
  --checkpoint_dir=/data/pretrained_models/${MODEL_NAME} \
  --model_dir=/data/pets/trained_model/${MODEL_NAME} # \
  # --run_once=True 


## Train (fine tune)

In [ ]:
%env MODEL_NAME=ssd_mobilenet_v1_coco_2018_01_28
#export MODEL_NAME='ssd_mobilenet_v1_coco_2018_01_28'
!python ${TF_OBJECT_DETECTION_API}/model_main.py \
    --alsologtostderr \
    --pipeline_config_path=/data/pets/trained_model/pipeline.config \
    --model_dir=/data/pets/trained_model/${MODEL_NAME} \
    --num_train_step=50000
    

# Legacy
## Eval

In [ ]:
%env MODEL_NAME=ssd_mobilenet_v1_coco_2018_01_28
#export MODEL_NAME='ssd_mobilenet_v1_coco_2018_01_28'
!python ${TF_OBJECT_DETECTION_API}/legacy/eval.py \
    --pipeline_config_path=/data/pets/trained_model/pipeline.config \
    --eval_dir=/data/pets/trained_model/${MODEL_NAME} \
    --checkpoint_dir=/data/pretrained_models/${MODEL_NAME}

## Train

In [ ]:
%env MODEL_NAME=ssd_mobilenet_v1_coco_2018_01_28
#export MODEL_NAME='ssd_mobilenet_v1_coco_2018_01_28'
!python ${TF_OBJECT_DETECTION_API}/legacy/train.py \
    --pipeline_config_path=/data/pets/trained_model/pipeline.config \
    --train_dir=/data/pets/trained_model  \
    --checkpoint_dir=/data/pretrained_models/${MODEL_NAME}

## Train using multiple GPU

In [ ]:
%env MODEL_NAME=ssd_mobilenet_v1_coco_2018_01_28
#export MODEL_NAME='ssd_mobilenet_v1_coco_2018_01_28'
!python ${TF_OBJECT_DETECTION_API}/legacy/train.py \
    --pipeline_config_path=/data/pets/trained_model/pipeline.config \
    --train_dir=/data/pets/trained_model  \
    --checkpoint_dir=/data/pretrained_models/${MODEL_NAME} \
    --worker_replicas=2 --num_clones=2 --ps_tasks=1